<a href="https://colab.research.google.com/github/Weirdo-001/aarushi/blob/main/Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers accelerate datasets timm opencv-python-headless -q
import transformers
print(transformers.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
4.56.2


In [2]:
import os

import cv2
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification,ViTFeatureExtractor,Trainer,TrainingArguments


In [3]:
from google.colab import drive
drive.mount('/content/drive')
patho = "/content/drive/MyDrive/archive/DFD_original_sequences"
pathm = "/content/drive/MyDrive/archive/DFD_manipulated_sequences"

sizei = 224
fskip = 60
maxfpv = 5
batchs = 16
epochs = 5


Mounted at /content/drive


In [4]:

!ls "/content/drive/MyDrive/archive"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DFD_manipulated_sequences  DFD_original_sequences  testingsample.mp4


In [5]:
import glob
import os
import cv2
from tqdm import tqdm

maxvpclass = 200
def extract_frames_from_videos(pathfolder,label,img_size=(sizei, sizei),frame_skip=fskip,max_frames=maxfpv):
    data, labels = [],[]
    fvideos = glob.glob(os.path.join(pathfolder,"**","*.mp4"),recursive=True)

    if maxvpclass:
        fvideos =fvideos[:maxvpclass]
    for filepath in tqdm(fvideos,desc=f"Processing {'Real' if label==0 else 'Fake'}"):
        cap = cv2.VideoCapture(filepath)
        countf = 0
        collectedf =0

        while cap.isOpened() and collectedf <max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            if countf%frame_skip== 0:
                resized = cv2.resize(frame,img_size)
                data.append(resized)
                labels.append(label)
                collectedf +=1
            countf+= 1
        cap.release()
    return data,labels



patho = "/content/drive/MyDrive/archive/DFD_original_sequences"
pathm = "/content/drive/MyDrive/archive/DFD_manipulated_sequences"

dreal,real_labels=extract_frames_from_videos(patho,0)
dfake,fake_labels= extract_frames_from_videos(pathm,1)
print(f"Extracted: {len(dreal)} real frames, {len(dfake)} fake frames")


Processing Fake: 100%|██████████| 200/200 [06:54<00:00,  2.07s/it]

Extracted: 998 real frames, 962 fake frames


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np

datall=np.array(dreal + dfake,dtype=np.float32)
labelsall= np.array(real_labels + fake_labels)

print("Final dataset shape:", datall.shape, labelsall.shape)
X_train,X_test,y_train,y_test=train_test_split(datall,labelsall,test_size=0.2, random_state=42,stratify=labelsall)

print("Train:",X_train.shape, " Test:",X_test.shape)
print("Class balance:",np.bincount(labelsall))


Final dataset shape: (1960, 224, 224, 3) (1960,)
Train: (1568, 224, 224, 3)  Test: (392, 224, 224, 3)
Class balance: [998 962]


In [7]:
from transformers import ViTImageProcessor
from torch.utils.data import Dataset
import torch

feature_extractor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

class DeepfakeDataset(Dataset):
    def __init__(self,images,labels,extractor):
        self.images =images
        self.labels =labels
        self.extractor =extractor

    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        img= self.images[idx]
        label =int(self.labels[idx])
        encoding = self.extractor(images=img,return_tensors="pt")
        encoding = {k: v.squeeze(0) for k,v in encoding.items()}
        encoding["labels"] = torch.tensor(label, dtype=torch.long)
        return encoding

train_dataset=DeepfakeDataset(X_train,y_train,feature_extractor)
test_dataset= DeepfakeDataset(X_test,y_test,feature_extractor)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [8]:
from transformers import ViTForImageClassification

modelvit = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=2,
    ignore_mismatched_sizes=True
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import os
os.environ["WANDB_DISABLED"] ="true"

from transformers import ViTImageProcessor,ViTForImageClassification,Trainer,TrainingArguments
import torch
from torch.utils.data import Dataset
from torchvision import transforms

transformertrain = transforms.Compose([
    transforms.RandomResizedCrop(224,scale=(0.8, 1.0)),transforms.RandomHorizontalFlip(),transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2,contrast=0.2,saturation=0.2,hue=0.1),transforms.ToTensor()
])

transformertest = transforms.Compose([
    transforms.Resize((224, 224)),transforms.ToTensor()
])

def compmetrics(eval_pred):
    logits,labels=eval_pred
    preds=np.argmax(logits, axis=-1)
    acc =np.mean(preds ==labels)
    tp =np.sum((preds==1)&(labels ==1))
    fp =np.sum((preds==1) &(labels ==0))
    fn =np.sum((preds==0)& (labels==1))
    precision = tp/(tp+fp+1e-9)
    recall =tp/(tp+fn+1e-9)
    f1 = 2*precision *recall/(precision +recall +1e-9)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}


from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./vit-deepfake",
    num_train_epochs=30,
    per_device_train_batch_size=16,per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    weight_decay=0.01,warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",
    logging_dir="./logs",logging_steps=50,
    fp16=True,save_total_limit=1
)

trainer = Trainer(
    model = modelvit,args=training_args,
    train_dataset=train_dataset,eval_dataset=test_dataset,
    tokenizer=feature_extractor,compute_metrics=compmetrics
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-1626891269.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.015200
100,0.048000
150,0.049000
200,0.047000
250,0.039500
300,0.029000
350,0.016900
400,0.012900
450,0.007600
500,0.002900


TrainOutput(global_step=750, training_loss=0.018196584338943163, metrics={'train_runtime': 614.2452, 'train_samples_per_second': 76.582, 'train_steps_per_second': 1.221, 'total_flos': 3.6452231914704077e+18, 'train_loss': 0.018196584338943163, 'epoch': 30.0})

In [17]:
metrics = trainer.evaluate()
print("Final Metrics:",metrics)

Final Metrics: {'eval_loss': 0.5095111131668091, 'eval_accuracy': 0.9209183673469388, 'eval_precision': 0.8965517241335146, 'eval_recall': 0.9479166666617297, 'eval_f1': 0.9215189868374941, 'eval_runtime': 2.4373, 'eval_samples_per_second': 160.836, 'eval_steps_per_second': 10.257, 'epoch': 30.0}


In [18]:
modelvit.save_pretrained("/content/drive/MyDrive/vit_deepfake_model")
feature_extractor.save_pretrained("/content/drive/MyDrive/vit_deepfake_model")

['/content/drive/MyDrive/vit_deepfake_model/preprocessor_config.json']

In [19]:
def predict_video_vit(video_path, modelvit,extractor,frame_skip=30,img_size=(sizei, sizei)):
    cap = cv2.VideoCapture(video_path)
    predictions = []
    countf = 0

    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break
        if countf % frame_skip == 0:
            resized = cv2.resize(frame, img_size)

            inputs= extractor(images=resized, return_tensors="pt")
            inputs={k:v.to(modelvit.device) for k,v in inputs.items()}
            outputs = modelvit(**inputs)
            pred = torch.softmax(outputs.logits,dim=1)[0][1].item()
            predictions.append(pred)
        countf+=1

    cap.release()

    if len(predictions) ==0:
        print("Not processed Anyy Frames")
        return
    avg_pred = np.mean(predictions)
    print(f"Prediction Score: {avg_pred:.2f} → {'FAKE' if avg_pred>0.5 else 'REAL'}")

predict_video_vit("/content/drive/MyDrive/archive/testingsample.mp4", modelvit, feature_extractor)


Prediction Score: 0.99 → FAKE
